# Simple tool to read the demo JSON data model file

In [1]:
import os
import json
import pandas as pd
import numpy as np
import requests

Note: Requires Pandas version +1.0

### Load the JSON file

In [2]:
try:
    # first try to load the file from your computer
    cwd = os.getcwd()
    file_path = os.path.relpath(r'..\demo_data\iea43_wra_data_model.json', cwd)
    with open(file_path) as json_file:
        meta_data = json.load(json_file)
    print("Loaded JSON file from local machine.")
except FileNotFoundError:
    # If the file isn't found, load it from GitHub
    url = "https://raw.githubusercontent.com/IEA-Task-43/digital_wra_data_standard/master/demo_data/iea43_wra_data_model.json"
    resp = requests.get(url)
    meta_data = json.loads(resp.text)
    print("Loaded JSON file from GitHub.")

Loaded JSON file from local machine.


### Show high level Data Model information

In [3]:
print('Author name:\t\t\t{}'.format(meta_data['author']))
print('Author from:\t\t\t{}'.format(meta_data['organisation']))
print('Date produced:\t\t\t{}'.format(meta_data['date']))
print('IEA Data Model version\t\t{}'.format(meta_data['version']))

print('Plant name:\t\t\t{}'.format(meta_data['plant_name']))
print('Plant type:\t\t\t{}'.format(meta_data['plant_type']))


Author name:			Stephen Holleran
Author from:			brightwind
Date produced:			2021-12-23
IEA Data Model version		1.3.0-2024.03
Plant name:			A Name of the Wind Farm
Plant type:			onshore_wind


### Show all the measurement locations

In [4]:
# print a table of the meas_locs parameters.
meas_locs = []
for meas_loc in meta_data['measurement_location']:
    meas_locs.append({
        'UUID': meas_loc['uuid'],
        'Name': meas_loc['name'],
        'Latitude [Decimal Degrees]': meas_loc['latitude_ddeg'],
        'Longitude [Decimal Degrees]': meas_loc['longitude_ddeg'],
        'Station Type': meas_loc['measurement_station_type_id'],
        'Notes': meas_loc['notes'],
        'Last Updated': meas_loc['update_at'],
        'Mast Type': meas_loc['mast_properties']['mast_geometry_id'],
        'Mast Height': meas_loc['mast_properties']['mast_height_m'],
        'Mast OEM': meas_loc['mast_properties']['mast_oem']
    })

meas_locs_df = pd.DataFrame(meas_locs)
meas_locs_df.set_index('Name', inplace=True)
display(meas_locs_df)

,UUID,Latitude [Decimal Degrees],Longitude [Decimal Degrees],Station Type,Notes,Last Updated,Mast Type,Mast Height,Mast OEM
Name,,,,,,,,,
Test_MM1,6858cf5c-24e0-40d4-955b-8aecbccba391,53.5,-8,mast,I can write anything I want here.,2020-04-18T18:13:00,lattice_triangle,78.5,A Mast Manufacturer


### Logger main configurations

In [5]:
logger_main_config = []
for meas_loc in meta_data['measurement_location']:
    for log_config in meas_loc['logger_main_config']:
        logger_main_config.append(log_config)

logger_main_config_df = pd.DataFrame(logger_main_config)
display(logger_main_config_df.set_index('logger_name'))

,logger_serial_number,logger_model_name,logger_id,logger_oem_id,logger_firmware_version,date_from,date_to,encryption_pin_or_key,enclosure_lock_details,data_transfer_details,offset_from_utc_hrs,sampling_rate_sec,averaging_period_minutes,timestamp_is_end_of_period,clock_is_auto_synced,logger_acquisition_uncertainty,uncertainty_k_factor,notes,update_at
logger_name,,,,,,,,,,,,,,,,,,,
AName_MM1,01002,Symphonie Plus3,4321,NRG Systems,3.2.3,2020-04-12T12:00:00,None,9876,combination lock PIN 54321,Emails to data@developername.com,-5,3,10,False,True,0.1,2,I can write anything I want here.,2020-04-18T18:13:00


### Some functions to pull out and format the measurement points set ups.

In [6]:
def _flatten_sensor_dict(sensor):
    """
        Flatten the sensor dictionary retrieved from jason
        assigning all the sub-dictionaries to the main dictionary.

        :param sensor: The sensor dictionary retrieved for a single configuration
                           option and meas_point id.
        :type sensor: dict
        :return: output
        :rtype: dict

    """
    output = {key: value for key, value in sensor.items() if (type(value) != list) or (value == {})}
    for key, value in zip(sensor.keys(), sensor.values()):
        if (type(value) == list):
            if key == 'calibration':
                value = {key + "_" + k: v for k, v in value[0].items()}
            output.update(value)
    return output

In [7]:
def rename_variables(input_dict, root_name):
    for var_to_rename in ['height_m', 'serial_number', 'update_at', 'notes']:
        if var_to_rename in list(input_dict.keys()):
            input_dict[ root_name + '_' + var_to_rename] = input_dict.pop(var_to_rename)
    return input_dict

In [8]:
def replace_none_date(input_dict):
    for date_str in ['date_from', 'date_to']:
        if input_dict[date_str] is None:
            input_dict[date_str] = '2100-12-31T00:00:00'
    return input_dict

In [9]:
def get_meas_points(meas_points):
    
    meas_points_flatten = []
    for meas_point in meas_points:
#         meas_point = _flatten_meas_point_dict(meas_point)
        log_meas_configs = sorted( meas_point['logger_measurement_config'], key=lambda i: i['date_from'])
        log_meas_configs = [replace_none_date(rename_variables(log_meas_config, 'log_meas_config')) for log_meas_config in log_meas_configs]
        sensors = [replace_none_date(rename_variables(_flatten_sensor_dict(sensor), 'sensor')) for sensor in meas_point['sensor']]
        if meas_point['mounting_arrangement'] is not None:
            mounting_arrangements = [replace_none_date(rename_variables(mntg_arrang, 'mounting_arrangement')) 
                                 for mntg_arrang in meas_point['mounting_arrangement']]
        else:
            mounting_arrangements = {}
        
        date_from = [log_meas_config['date_from'] for log_meas_config in log_meas_configs]
        date_to = [log_meas_config['date_to'] for log_meas_config in log_meas_configs]
        for sensor in sensors:
            date_from.append(sensor['date_from'])
            date_to.append(sensor['date_to'])
        for mntg_arrang in mounting_arrangements:
            date_from.append(mntg_arrang['date_from'])
            date_to.append(mntg_arrang['date_to'])
        
        date_from.extend(date_to)
        dates = np.unique(date_from)
        for i in range(len(dates)-1): 
            good_log_meas_config = {}
            for log_meas_config in log_meas_configs:
                if (log_meas_config['date_from'] <= dates[i]) & (log_meas_config['date_to'] > dates[i]):
                    good_log_meas_config = log_meas_config.copy()
            if good_log_meas_config != {}:
                for sensor in sensors: 
                    if (sensor['date_from'] <= dates[i]) & (sensor['date_to'] > dates[i]) :
                        good_log_meas_config.update(sensor)
                for mntg_arrang in mounting_arrangements:
                    if (mntg_arrang['date_from'] <= dates[i]) & (mntg_arrang['date_to'] > dates[i]) :
                        good_log_meas_config.update(mntg_arrang)
                good_log_meas_config['date_to'] = dates[i+1]
                good_log_meas_config['date_from'] = dates[i]
                good_log_meas_config.update(meas_point)
                del good_log_meas_config['logger_measurement_config']
                del good_log_meas_config['sensor'] 
                meas_points_flatten.append(good_log_meas_config)
    return meas_points_flatten 

In [10]:
def _format_sensor_table(meas_points, table_type='full'):
    
    if table_type == 'full':
        header = ['name', 'measurement_units', 'oem',
                  'height_m', 'boom_orientation_deg', 'vane_dead_band_orientation_deg',
                  'date_from', 'date_to', 'connection_channel', 'log_meas_config_height_m', 'slope', 'offset', 'calibration_slope',
                  'calibration_offset']
        header_for_report = ['Instrument Name', 'Units', 'Sensor OEM',
                        'Height [m]', 'Boom Orient. [deg, mag N]', 'Dead Band Orient. [deg, mag N]',
                        'Date From', 'Date To', 'Logger Channel', 'Logger Stated Height [m]', 'Logger Slope', 'Logger Offset', 'Calibration Slope',
                        'Calibration Offset']
    elif table_type == 'meas_points':
        header = ['name', 'measurement_type_id', 'height_m', 'boom_orientation_deg']
        header_for_report = ['Instrument Name', 'Measurement Type', 'Height [m]', 'Boom Orient. [deg, mag N]']    
    elif table_type == 'speed_info':
        header = ['name', 'measurement_units', 'oem', 'model', 'sensor_serial_number',
                  'height_m', 'boom_orientation_deg', 
                  'date_from', 'date_to', 'slope', 'offset', 'calibration_slope',
                  'calibration_offset', 'measurement_type_id']
        header_for_report = ['Instrument Name', 'Units', 'Sensor Make', 'Sensor Model', 'Serial No',
                             'Height [m]', 'Boom Orient. [deg, mag N]',
                             'Date From', 'Date To', 'Logger Slope', 'Logger Offset', 'Calibration Slope',
                             'Calibration Offset', 'measurement_type_id']
    elif table_type == 'direction_info':
        header = ['name', 'measurement_units', 'oem', 'model', 'sensor_serial_number',
                  'height_m', 'boom_orientation_deg', 'vane_dead_band_orientation_deg', 
                  'date_from', 'date_to', 'offset', 'measurement_type_id']
        header_for_report = ['Instrument Name', 'Units', 'Sensor Make', 'Sensor Model', 'Serial No',
                             'Height [m]', 'Boom Orient. [deg, mag N]', 'Dead Band Orient. [deg, mag N]',
                             'Date From', 'Date To', 'Logger Offset', 'measurement_type_id']
    
    sensors_table_report = pd.DataFrame(meas_points)

    if any(elem not in sensors_table_report.columns for elem in header):
        ind_to_remove = [ind for ind, elem in enumerate(header) if elem not in sensors_table_report.columns]
        del header[ind_to_remove[0]]
        del header_for_report[ind_to_remove[0]]
    
    sensors_table_report = pd.DataFrame(sensors_table_report[header])
    if table_type == 'speed_info':
        sensors_table_report = sensors_table_report[sensors_table_report['measurement_type_id'] == 'wind_speed']
        del sensors_table_report['measurement_type_id']
    if table_type == 'direction_info':
        sensors_table_report = sensors_table_report[sensors_table_report['measurement_type_id'] == 'wind_direction']
        del sensors_table_report['measurement_type_id']
    
    if 'date_from' in sensors_table_report.columns:
        sensors_table_report['date_from'] = pd.to_datetime(sensors_table_report['date_from'].values.astype(str), 
                                                           format='%Y-%m-%dT%H:%M:%S').strftime("%d-%b-%Y")
    if 'date_to' in sensors_table_report.columns:
        sensors_table_report['date_to'] = pd.to_datetime(sensors_table_report['date_to'].values.astype(str), 
                                                         format='%Y-%m-%dT%H:%M:%S').strftime("%d-%b-%Y")

    sensors_table_report = sensors_table_report.replace({np.nan: '-', 'NaT': '-', '31-Dec-2100':'-'})
    sensors_table_report.rename(columns={k: h for k, h in zip(header, header_for_report)}, inplace=True)
    index_name = 'Instrument Name'
    sensors_table_report = sensors_table_report.set_index(index_name)
    
    return sensors_table_report

### The main measurment points

In [11]:
for meas_loc in meta_data['measurement_location']:  
    logger_meas_configs = get_meas_points(meas_loc['measurement_point'])
    sensors_table = _format_sensor_table(logger_meas_configs, table_type='meas_points')
    display(sensors_table.drop_duplicates())

,Measurement Type,Height [m],"Boom Orient. [deg, mag N]"
Instrument Name,,,
Spd_80.1_315,wind_speed,80.1,315.0
Spd_80mSE,wind_speed,80.2,135.0
Spd_60mNW,wind_speed,60.1,315.0
Spd_60mSE,wind_speed,60.2,135.0
Spd_40mNW,wind_speed,40.1,315.0
Spd_30mNW,wind_speed,30.1,315.0
Spd_40mSE,wind_speed,40.2,135.0
Dir_76mNW,wind_direction,76.1,315.0
Dir_56mNW,wind_direction,56.1,315.0


### More details on each measurement point

In [12]:
for meas_loc in meta_data['measurement_location']:
    logger_meas_configs = get_meas_points(meas_loc['measurement_point'])
    sensors_table = _format_sensor_table(logger_meas_configs)
    display(sensors_table)

,Sensor OEM,Height [m],"Boom Orient. [deg, mag N]","Dead Band Orient. [deg, mag N]",Date From,Date To,Logger Channel,Logger Stated Height [m],Logger Slope,Logger Offset,Calibration Slope,Calibration Offset
Instrument Name,,,,,,,,,,,,
Spd_80.1_315,Thies,80.1,315.0,-,12-Apr-2020,15-Apr-2020,CH1,80,0.04573,0.2419,0.04573,0.2419
Spd_80.1_315,Thies,80.1,315.0,-,15-Apr-2020,-,CH1,80,0.04573,0.2491,0.04573,0.2419
Spd_80mSE,Thies,80.2,135.0,-,12-Apr-2020,18-Apr-2020,CH2,80,0.04568,0.2487,0.04568,0.2487
Spd_80mSE,Thies,80.2,135.0,-,18-Apr-2020,-,CH2,80,0.04575,0.2497,0.04575,0.2497
Spd_60mNW,Thies,60.1,315.0,-,12-Apr-2020,-,CH3,60,0.04666,0.2416,0.04666,0.2416
Spd_60mSE,Thies,60.2,135.0,-,12-Apr-2020,-,CH4,60,0.04777,0.2417,0.04777,0.2417
Spd_40mNW,Thies,40.1,315.0,-,12-Apr-2020,18-Apr-2020,CH5,60,0.04888,0.2418,0.04888,0.2418
Spd_40mNW,Thies,40.1,315.0,-,18-Apr-2020,-,CH14,40,0.04888,0.2418,0.04888,0.2418
Spd_30mNW,Thies,30.1,315.0,-,12-Apr-2020,18-Apr-2020,CH6,30,0.04999,0.2419,0.04999,0.2419


### Anemometer (or wind speed) specific table

In [13]:
for meas_loc in meta_data['measurement_location']:    
    sensors_table = _format_sensor_table(logger_meas_configs, table_type='speed_info')
    display(sensors_table.drop_duplicates())

,Sensor Make,Sensor Model,Serial No,Height [m],"Boom Orient. [deg, mag N]",Date From,Date To,Logger Slope,Logger Offset,Calibration Slope,Calibration Offset
Instrument Name,,,,,,,,,,,
Spd_80.1_315,Thies,4.3351.10.000,09183000,80.1,315.0,12-Apr-2020,15-Apr-2020,0.04573,0.2419,0.04573,0.2419
Spd_80.1_315,Thies,4.3351.10.000,09183000,80.1,315.0,15-Apr-2020,-,0.04573,0.2491,0.04573,0.2419
Spd_80mSE,Thies,4.3351.10.000,09183001,80.2,135.0,12-Apr-2020,18-Apr-2020,0.04568,0.2487,0.04568,0.2487
Spd_80mSE,Thies,4.3351.10.000,09183023,80.2,135.0,18-Apr-2020,-,0.04575,0.2497,0.04575,0.2497
Spd_60mNW,Thies,4.3351.10.000,09183002,60.1,315.0,12-Apr-2020,-,0.04666,0.2416,0.04666,0.2416
Spd_60mSE,Thies,4.3351.10.000,09183003,60.2,135.0,12-Apr-2020,-,0.04777,0.2417,0.04777,0.2417
Spd_40mNW,Thies,4.3351.10.000,09183004,40.1,315.0,12-Apr-2020,18-Apr-2020,0.04888,0.2418,0.04888,0.2418
Spd_40mNW,Thies,4.3351.10.000,09183004,40.1,315.0,18-Apr-2020,-,0.04888,0.2418,0.04888,0.2418
Spd_30mNW,Thies,4.3351.10.000,09183005,30.1,315.0,12-Apr-2020,18-Apr-2020,0.04999,0.2419,0.04999,0.2419


### Wind Vane (or wind direction) specific table

In [14]:
for meas_loc in meta_data['measurement_location']:    
    sensors_table = _format_sensor_table(logger_meas_configs, table_type='direction_info')
    display(sensors_table.drop_duplicates())

,Sensor Make,Sensor Model,Serial No,Height [m],"Boom Orient. [deg, mag N]","Dead Band Orient. [deg, mag N]",Date From,Date To,Logger Offset
Instrument Name,,,,,,,,,
Dir_76mNW,NRG,#200P,01234589,76.1,315.0,315.0,12-Apr-2020,-,-
Dir_56mNW,NRG,#200P,01234567,56.1,315.0,315.0,12-Apr-2020,18-Apr-2020,-
Dir_56mNW,NRG,#200P,01234588,56.1,315.0,135.0,18-Apr-2020,-,-


In [15]:
logger_meas_configs

[{'slope': 0.04573,
  'offset': 0.2419,
  'sensitivity': None,
  'measurement_units_id': 'm/s',
  'connection_channel': 'CH1',
  'logger_stated_boom_orientation_deg': 310,
  'date_from': '2020-04-12T12:00:00',
  'date_to': '2020-04-15T00:00:00',
  'column_name': [{'column_name': 'CH1Avg',
    'statistic_type_id': 'avg',
    'is_ignored': False,
    'notes': 'I can write anything I want here.',
    'update_at': '2020-04-18T18:13:00'},
   {'column_name': 'CH1SD',
    'statistic_type_id': 'sd',
    'is_ignored': False,
    'notes': 'I can write anything I want here.',
    'update_at': '2020-04-18T18:13:00'},
   {'column_name': 'CH1Min',
    'statistic_type_id': 'min',
    'is_ignored': False,
    'notes': 'I can write anything I want here.',
    'update_at': '2020-04-18T18:13:00'},
   {'column_name': 'CH1Max',
    'statistic_type_id': 'max',
    'is_ignored': False,
    'notes': 'I can write anything I want here.',
    'update_at': '2020-04-18T18:13:00'},
   {'column_name': 'CH1Ti30sec',
